## text inversion

In [ ]:
!accelerate launch textual_inversion.py \
  --pretrained_model_name_or_path="stabilityai/stable-diffusion-2-1" \
  --train_data_dir="./minhwa_dataset/jpg/" \
  --learnable_property="style" \
  --placeholder_token="<minhwa-style>" \
  --initializer_token="painting" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --max_train_steps=3000 \
  --learning_rate=5.0e-04 \
  --scale_lr \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --output_dir="minhwa_textual_inversion"

## lora

In [ ]:
import os
import json

json_dir = '../json/' # json 파일이 있는 경로
img_dir = './'       # jpg 파일이 있는 현재 경로
output_file = os.path.join(img_dir, 'metadata.jsonl')

with open(output_file, 'w', encoding='utf-8') as outfile:
    for json_filename in os.listdir(json_dir):
        if json_filename.endswith('.json'):
            json_path = os.path.join(json_dir, json_filename)
            
            with open(json_path, 'r', encoding='utf-8') as infile:
                data = json.load(infile)
                
                image_filename = data['images']['identifier']

                # --- [핵심 수정 부분] ---
                # 실제 이미지 파일의 전체 경로 생성
                image_filepath = os.path.join(img_dir, image_filename)
                
                # 파일이 실제로 존재하는 경우에만 메타데이터를 생성
                if not os.path.exists(image_filepath):
                    print(f"\n경고: 이미지 파일을 찾을 수 없습니다. 건너뜁니다: {image_filename}")
                    continue # 다음 파일로 넘어감
                # --- [수정 끝] ---

                caption = data['images']['caption_kr']
                
                # 최종 프롬프트 구성 (스타일 + 내용)
                # 중요: 아래 '언어 문제' 항목을 꼭 읽어보세요!
                prompt = f"minhwa painting style, {caption}"
                
                # metadata.jsonl에 쓸 데이터
                entry = {
                    "file_name": image_filename,
                    "text": prompt
                }
                
                # 파일에 한 줄씩 쓰기
                outfile.write(json.dumps(entry, ensure_ascii=False) + '\n')

print(f"'{output_file}' 생성이 완료되었습니다.")

In [ ]:
!accelerate launch train_text_to_image_lora.py \
    --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \
    --dataset_name="./minhwa_dataset/jpg/" \
    --resolution=512 \
    --center_crop \
    --random_flip \
    --train_batch_size=1 \
    --gradient_accumulation_steps=4 \
    --num_train_epochs=10 \
    --learning_rate=1e-04 \
    --lr_scheduler="cosine" \
    --lr_warmup_steps=0 \
    --output_dir="minhwa_lora_model" \
    --rank=16 \
    --mixed_precision="no"

In [4]:
import os
import json
from deep_translator import GoogleTranslator
from tqdm import tqdm

# --- 설정 ---
json_dir = './minhwa_dataset/json/'
img_dir = './minhwa_dataset/jpg/'
output_file = os.path.join(img_dir, 'metadata_withcategory.jsonl')
# --- 설정 끝 ---

# --- 카테고리/서브카테고리 매핑 (1단계에서 정의) ---
# CATEGORY_MAP = {
#     1: "화조도", 2: "초충도", 3: "소과도", 4: "모란도", 5: "십장생도",
#     6: "책가도", 7: "문자도", 8: "어해도", 9: "호도", 10: "용도",
#     11: "영수도", 12: "인물도", 13: "산수도"
# }
CATEGORY_MAP = {
    1: "Hwajodo",
    2: "Chochungdo",
    3: "Sogwado",
    4: "Morando",
    5: "Sipjangsaengdo",
    6: "Chaekgado",
    7: "Munjado",
    8: "Eohaedo",
    9: "Hodo",
    10: "Yongdo",
    11: "Yeongsudo",
    12: "Inmuldo",
    13: "Sansudo"
}

# SUBCATEGORY_MAP = {
#     1: "꽃", 2: "새", 3: "풀", 4: "곤충", 5: "채소",
#     6: "과일", 7: "모란", 8: "십장생", 9: "문방사우", 10: "책거리",
#     11: "부귀 문자", 12: "효제 문자", 13: "어류", 14: "갑각류 외", 15: "호랑이",
#     16: "용", 17: "상상동물", 18: "동자도", 19: "신선도", 20: "풍속도",
#     21: "산", 22: "석", 23: "수지", 24: "산수,풍경"
# }
SUBCATEGORY_MAP = {
    1: "Flowers",
    2: "Birds",
    3: "Grass / Plants",
    4: "Insects",
    5: "Vegetables",
    6: "Fruits",
    7: "Peony",
    8: "The Ten Symbols of Longevity",
    9: "The Four Friends of the Study",
    10: "Chaekgeori (Books and Things)",
    11: "Characters of Wealth and Nobility",
    12: "Characters of Filial Piety",
    13: "Fish",
    14: "Crustaceans and others",
    15: "Tiger",
    16: "Dragon",
    17: "Mythical Creatures",
    18: "Paintings of Boys / Children",
    19: "Paintings of Taoist Immortals",
    20: "Genre Paintings",
    21: "Mountain",
    22: "Rock / Stone",
    23: "Waterside / Trees",
    24: "Landscape"
}
# --- 매핑 끝 ---

translator = GoogleTranslator(source='ko', target='en')
json_files = [f for f in os.listdir(json_dir) if f.endswith('.json')]

print(f"총 {len(json_files)}개의 json 파일에 대한 번역 및 전처리를 시작합니다.")

with open(output_file, 'w', encoding='utf-8') as outfile:
    for json_filename in tqdm(json_files, desc="Processing files"):
        json_path = os.path.join(json_dir, json_filename)
        
        try:
            with open(json_path, 'r', encoding='utf-8') as infile:
                data = json.load(infile)
                
                image_info = data['images']
                image_filename = image_info['identifier']

                # --- [핵심 수정 부분] ---
                # 실제 이미지 파일의 전체 경로 생성
                image_filepath = os.path.join(img_dir, image_filename)
                
                # 파일이 실제로 존재하는 경우에만 메타데이터를 생성
                if not os.path.exists(image_filepath):
                    print(f"\n경고: 이미지 파일을 찾을 수 없습니다. 건너뜁니다: {image_filename}")
                    continue # 다음 파일로 넘어감
                # --- [수정 끝] ---


                korean_caption = image_info['caption_kr']
                
                # --- [수정된 부분 시작] ---
                
                # 1. 카테고리/서브카테고리 번호 가져오기
                category_num = image_info.get('category')
                subcategory_num = image_info.get('subcategory')
                
                # 2. 번호를 텍스트로 변환 (딕셔너리 사용)
                # .get()을 사용하여 키가 없는 경우에도 오류가 나지 않도록 처리
                category_kr = CATEGORY_MAP.get(int(category_num), "")
                subcategory_kr = SUBCATEGORY_MAP.get(int(subcategory_num), "")

                # 3. 캡션, 카테고리, 서브카테고리 모두 번역
                # 하나의 리스트로 묶어 한 번에 번역 요청하면 더 효율적입니다.
                texts_to_translate = [korean_caption, category_kr, subcategory_kr]
                translations = translator.translate_batch(texts_to_translate)
                english_caption, category_en, subcategory_en = translations

                # 4. 최종 영어 프롬프트 구성 (스타일 + 카테고리 + 서브카테고리 + 내용)
                # 프롬프트는 쉼표로 구분하는 것이 일반적이고 효과적입니다.
                prompt_parts = ["a minhwa painting"]
                if category_en:
                    prompt_parts.append(category_en) # 예: "landscape painting"
                if subcategory_en:
                    prompt_parts.append(subcategory_en) # 예: "mountain"
                if english_caption:
                    prompt_parts.append(english_caption) # 예: "Mountains of blue and gray overlap to fill the screen."
                
                prompt = ", ".join(prompt_parts)
                
                # --- [수정된 부분 끝] ---

                entry = {
                    "file_name": image_filename,
                    "text": prompt
                }
                
                outfile.write(json.dumps(entry, ensure_ascii=False) + '\n')

        except Exception as e:
            print(f"\n파일 처리 중 오류 발생: {json_filename}, 오류: {e}")
            continue

print(f"\n'{output_file}' 생성이 완료되었습니다.")
print("이제 카테고리 정보가 포함된 프롬프트로 모델 학습을 진행할 수 있습니다.")

총 650개의 json 파일에 대한 번역 및 전처리를 시작합니다.


Processing files:   1%|          | 4/650 [00:07<22:12,  2.06s/it]


경고: 이미지 파일을 찾을 수 없습니다. 건너뜁니다: 김민회_산수도_수지7(활엽수).jpg


Processing files:   8%|▊         | 55/650 [01:29<13:34,  1.37s/it]


경고: 이미지 파일을 찾을 수 없습니다. 건너뜁니다: 김채현_산수도_수지26(침엽수).jpg


Processing files: 100%|██████████| 650/650 [12:10<00:00,  1.12s/it]


'./minhwa_dataset/jpg/metadata_withcategory.jsonl' 생성이 완료되었습니다.
이제 카테고리 정보가 포함된 프롬프트로 모델 학습을 진행할 수 있습니다.


In [ ]:
!accelerate launch train_text_to_image_lora.py \
    --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \
    --dataset_name="./minhwa_dataset/jpg/" \
    --resolution=512 \
    --center_crop \
    --random_flip \
    --train_batch_size=1 \
    --gradient_accumulation_steps=4 \
    --num_train_epochs=15 \
    --learning_rate=1e-04 \
    --lr_scheduler="cosine" \
    --lr_warmup_steps=0 \
    --output_dir="minhwa_lora_model_withcategory_r32" \
    --rank=32 \
    --mixed_precision="no" \
    --checkpointing_steps=162 

--checkpointing_steps=162  : 648/4 (1 epoch!)

In [ ]:
!accelerate launch train_text_to_image_lora.py \
    --pretrained_model_name_or_path="runwayml/stable-diffusion-v1-5" \
    --dataset_name="./minhwa_dataset/jpg/" \
    --resolution=512 \
    --center_crop \
    --random_flip \
    --train_batch_size=1 \
    --gradient_accumulation_steps=4 \
    --num_train_epochs=20 \
    --learning_rate=5e-05 \
    --lr_scheduler="cosine" \
    --lr_warmup_steps=0 \
    --output_dir="minhwa_lora_model_withcategory_r64" \
    --rank=64 \
    --mixed_precision="no" \
    --checkpointing_steps=162 

### text inversion with several keywords

- num_vectors : 4~16 

In [ ]:
!accelerate launch textual_inversion.py \
  --num_vectors=16 \
  --pretrained_model_name_or_path="stable-diffusion-v1-5/stable-diffusion-v1-5" \
  --train_data_dir="./minhwa_dataset/jpg/" \
  --learnable_property="style" \
  --placeholder_token="<minhwa-style>" \
  --initializer_token="painting" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --max_train_steps=3000 \
  --learning_rate=5.0e-04 \
  --scale_lr \
  --lr_scheduler="cosine" \
  --lr_warmup_steps=100 \
  --output_dir="minhwa_textual_inversion_several/minhwa_style/"

In [ ]:
!accelerate launch textual_inversion.py \
  --num_vectors=16 \
  --pretrained_model_name_or_path="stable-diffusion-v1-5/stable-diffusion-v1-5" \
  --train_data_dir="./minhwa_dataset/jpg_catagory/tiger/" \
  --learnable_property="object" \
  --placeholder_token="<minhwa-tiger>" \
  --initializer_token="painting" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --max_train_steps=3000 \
  --learning_rate=5.0e-04 \
  --scale_lr \
  --lr_scheduler="cosine" \
  --lr_warmup_steps=100 \
  --output_dir="minhwa_textual_inversion_several/minhwa_tiger/"

In [ ]:
!accelerate launch textual_inversion.py \
  --num_vectors=16 \
  --pretrained_model_name_or_path="stable-diffusion-v1-5/stable-diffusion-v1-5" \
  --train_data_dir="./minhwa_dataset/jpg_catagory/landscape/" \
  --learnable_property="object" \
  --placeholder_token="<minhwa-landscape>" \
  --initializer_token="painting" \
  --resolution=512 \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --max_train_steps=3000 \
  --learning_rate=5.0e-04 \
  --scale_lr \
  --lr_scheduler="cosine" \
  --lr_warmup_steps=100 \
  --output_dir="minhwa_textual_inversion_several/minhwa_landscape/"